In [15]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
import statsmodels.api as sm
from scipy import stats  

In [17]:
filepath = '../raw_data/productivity_data/total_factor_productivity.csv'
tfp_disp_df = pd.read_csv(filepath, encoding='latin1')

In [18]:
filepath = '../processed_data/tfp_four_digit_pivoted.csv'
manufacturing_tfp = pd.read_csv(filepath)

In [20]:
filepath = '../raw_data/dynamism_data/bds2022_vcn4.csv'
bds2022_vcn4 = pd.read_csv(filepath, encoding='latin1')

In [21]:
tfp_disp_df['naics4'] = tfp_disp_df['naics4'].astype(str)
bds2022_vcn4['vcnaics4'] = bds2022_vcn4['vcnaics4'].astype(str)

temp_df = pd.merge(
	tfp_disp_df, 
	bds2022_vcn4, 
	how='inner', 
	left_on=['naics4', 'year'], 
	right_on=['vcnaics4', 'year']
)

In [22]:
temp_df['naics4'] = pd.to_numeric(temp_df['naics4'], errors='raise').astype('int64')
manufacturing_tfp['NAICS'] = pd.to_numeric(manufacturing_tfp['NAICS'], errors='raise').astype('int64')

temp_df['year'] = temp_df['year'].astype(int)
manufacturing_tfp['year'] = pd.to_numeric(manufacturing_tfp['year'], errors='coerce').astype('Int64')
manufacturing_tfp = manufacturing_tfp.dropna(subset=['year'])
manufacturing_tfp['year'] = manufacturing_tfp['year'].astype(int)

merged_df = pd.merge(
    temp_df,
    manufacturing_tfp,
    how='inner',
    left_on = ['naics4', 'year'],
    right_on= ['NAICS',  'year']
)

In [23]:
merged_df['tfp_pct_change'] = pd.to_numeric(
    merged_df['tfp_pct_change'],
    errors='coerce'
)
merged_df['tfp_index_2017'] = pd.to_numeric(
    merged_df['tfp_index_2017'],
    errors='coerce')

In [25]:
columns_to_convert = [
    # Dispersion measures
    'd9990', 'd1001', 'd9990*', 'd1001*',
    
    # Entry/exit counts
    'estabs_entry', 'estabs_exit',
    'firmdeath_firms', 'firmdeath_estabs', 'firmdeath_emp',
    
    # Entry/exit rates  
    'estabs_entry_rate', 'estabs_exit_rate',
    
    # Job flow counts
    'job_creation_births', 'job_creation_continuers',
    'job_destruction_deaths', 'job_destruction_continuers',
    
    # Job flow rates
    'job_creation_rate_births', 'job_destruction_rate_deaths'
]

print("Converting object columns to numeric...")
for col in columns_to_convert:
    if col in merged_df.columns and merged_df[col].dtype == 'object':
        merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
        nan_count = merged_df[col].isna().sum()
        if nan_count > 0:
            print(f"  {col}: converted ({nan_count} NaN values created)")
        else:
            print(f"  {col}: converted successfully")

Converting object columns to numeric...
  d9990: converted (17 NaN values created)
  d1001: converted (16 NaN values created)
  d9990*: converted (8 NaN values created)
  d1001*: converted (8 NaN values created)
  estabs_entry: converted successfully
  estabs_exit: converted successfully
  firmdeath_firms: converted (4 NaN values created)
  firmdeath_estabs: converted (4 NaN values created)
  firmdeath_emp: converted (4 NaN values created)
  estabs_entry_rate: converted successfully
  estabs_exit_rate: converted successfully
  job_creation_births: converted successfully
  job_creation_continuers: converted successfully
  job_destruction_deaths: converted successfully
  job_destruction_continuers: converted successfully
  job_creation_rate_births: converted successfully
  job_destruction_rate_deaths: converted successfully


In [26]:
agg_dict = {
   # DISPERSION MEASURES - AVERAGE within 2-year window
   'sd': 'mean',
   'd7525': 'mean',
   'd9010': 'mean',
   'd9990': 'mean',
   'd1001': 'mean',
   'sd*': 'mean',
   'd7525*': 'mean',
   'd9010*': 'mean',
   'd9990*': 'mean',
   'd1001*': 'mean',

   # TFP metrics
   'tfp_pct_change': 'mean',     # average % change over the 2-year window
   'tfp_index_2017': 'last',     # index level at the end of the window

   # STOCK VARIABLES - LAST value (end of period snapshot)
   'firms': 'last',
   'estabs': 'last',
   'emp': 'last',
   'denom': 'last',

   # ENTRY/EXIT COUNTS - SUM across 2 years
   'estabs_entry': 'sum',
   'estabs_exit': 'sum',
   'firmdeath_firms': 'sum',
   'firmdeath_estabs': 'sum',
   'firmdeath_emp': 'sum',

   # ENTRY/EXIT RATES - AVERAGE across 2 years
   'estabs_entry_rate': 'mean',
   'estabs_exit_rate': 'mean',

   # JOB FLOW COUNTS - SUM across 2 years
   'job_creation': 'sum',
   'job_creation_births': 'sum',
   'job_creation_continuers': 'sum',
   'job_destruction': 'sum',
   'job_destruction_deaths': 'sum',
   'job_destruction_continuers': 'sum',
   'net_job_creation': 'sum',

   # JOB FLOW RATES - AVERAGE across 2 years
   'job_creation_rate': 'mean',
   'job_creation_rate_births': 'mean',
   'job_destruction_rate': 'mean',
   'job_destruction_rate_deaths': 'mean',
   'net_job_creation_rate': 'mean',
   'reallocation_rate': 'mean',
}